In [ ]:
import pandas as pd
from sklearn.ensemble.forest import RandomForestRegressor
import numpy as np

from datetime import datetime

In [ ]:
from tick_funcs import process, tickdata_on, transformations

df = process(
    tickdata_on(day=1, month=6, year=2021),
    tickdata_on(day=2, month=6, year=2021),
    tickdata_on(day=3, month=6, year=2021),
    tickdata_on(day=4, month=6, year=2021),
    tickdata_on(day=5, month=6, year=2021),
    tickdata_on(day=6, month=6, year=2021),
    tickdata_on(day=7, month=6, year=2021),
    tickdata_on(day=8, month=6, year=2021),
    tickdata_on(day=9, month=6, year=2021),
    tickdata_on(day=10, month=6, year=2021),
    # enrich with features
    **transformations
)
#df['timesince'] = pd.Series(pd.DatetimeIndex(df.timesince)-pd.DatetimeIndex(df.timesince.shift(1)),index=df.index).apply(lambda val: val.microseconds)

df.timestamp = pd.to_datetime(df.timestamp)

target = ['midPointDelta']
ignore = ['symbol', 'timestamp']


df.dropna(inplace=True)
df = df[df['midPointDelta'] < 400]
df = df[df['midPointDelta'] > -400]
df = df[df['bidSize'] != 1]
df = df[df['askSize'] != 1]


#df['ts'] = pd.to_datetime(df.timestamp)

df.sort_index(inplace=True)



In [ ]:
tdf = pd.read_csv('/home/rory/dev/tick-capture-dev/10-6-2021/XBTUSD_tick.csv')
#tdf.query('updateType == "Trade"')
tdf

In [ ]:
from datetime import timezone
from datetime import timedelta
df.timesince[df.index >= datetime.now(timezone.utc) - timedelta(hours=1)].plot()


# updates in second, minute,
df['updates_per_half_min'] = df.askPrice.rolling('30s').count()
df

In [ ]:
target = ['midPointDelta']
ignore = ['symbol', 'timestamp']

features = list(filter(lambda col: col not in target + ignore, df.columns))

num_lags = 3

lag_columns = []
for name in features:
    for i in range(1,1+num_lags):
        if '_lag_' not in name:
            df[f'{name}_lag_{i}'] = df[name].shift(periods=i, freq='s')


df


In [ ]:
df

In [ ]:
df.query( f'-50 < midPointDelta < 50').midPointDelta.hist(bins=100)

In [ ]:
df.bidSize.shift(freq='s', periods=)

In [ ]:
X = df[features].values
y = df[target].values

In [ ]:
features

In [ ]:
train_size = int(len(X) * 0.60)
X_train, X_test = X[0:train_size], X[train_size:len(X)]
y_train, y_test = y[0:train_size], y[train_size:len(X)]
print('Observations: %d' % (len(X)))
print('Training Observations: %d' % (len(X_train)))
print('Testing Observations: %d' % (len(X_test)))

In [ ]:
for num in {90, 100, 200}:
    rf = RandomForestRegressor(n_estimators=num)
    rf.fit(X=X_train, y=y_train)
    score = rf.score(X_test, y_test)
    print(f'score={score}, n_estimators={num}')

In [ ]:
rf.score(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x=rf.predict(X_test), y=y_test)

In [ ]:
print(GradientBoostingRegressor.__doc__)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)
gbr.score(X_test, y_test)

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

params = { 
    'n_estimators': 10000,
    'learning_rate': 0.01,
    'max_features': 30, 
    'max_depth': 4,
    'random_state': 0
}

parameter_space = {
    'learning_rate': [0.1,0.2,0.3],
    'max_depth': [3,4,5,6,8,10],
    'min_samples_split': [5],
    'n_estimators': [50,60,70,80,90,100]
}

#base_estimator = GradientBoostingRegressor()
#gb_reg.fit(X_train, y_train)
base_learner = GradientBoostingRegressor()
sh = GridSearchCV(base_learner, parameter_space, n_jobs=8, pre_dispatch=16)
sh.fit(X_train, y_train)

In [ ]:



with open('./lib/models/XBTUSD_tick.pickle', 'wb') as picklefile:
    picklefile.write(pickle.dumps(sh))
with open('./etc/models/XBTUSD_tick.json', 'w') as jsonfile:
    jsonfile.write(json.dumps(parameter_space))



In [ ]:
import pickle
import json

In [ ]:
params = {
    'learning_rate': 0.1,
    'max_depth': 6,
    'min_samples_split': 100,
    'n_estimators': 50
}
gb = GradientBoostingRegressor(**params)
gb.fit(X_train, y_train)



In [ ]:
gb.score(X_test, y_test)

In [ ]:
from sklearn import svm

sv = svm.SVR()
sv.fit(X_train, y_train)
